In [14]:
#Importing the required libraries
import pandas as pd
import numpy as np
import math
from datetime import date
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders

In [15]:
#Reading the sales data and inventory data into dataframes
sales_data=pd.read_excel('Zepto Case Study.xlsx',sheet_name='Sales Data')
inventory_data=pd.read_excel('Zepto Case Study.xlsx',sheet_name='Inventory Data')

In [16]:
#Grouping the data into product level to extrac the total sales, sales start date and sales end date
product_sales=sales_data.groupby(['product_variant_id']).agg(total_sales=('quantity_sold','sum'),start_date=('business_date','min'),end_date=('business_date','max')).reset_index()

In [17]:
#Caluculating the average sales perday for each SKU(Rate of Sales)
product_sales['no_days']=((product_sales['end_date']-product_sales['start_date']).dt.days)+1
product_sales['avg_sales']=product_sales['total_sales']/product_sales['no_days']

In [18]:
product_sales

,product_variant_id,total_sales,start_date,end_date,no_days,avg_sales
0,00610840-fed1-458a-9d1b-bb0cf05fbab0,11,1989-11-18,1990-01-01,45,0.244444
1,006a159f-44c1-46c4-bf62-232ce0a82276,42,1989-11-12,1989-12-29,48,0.875000
2,00a4c018-3370-4d6e-8ee9-08d9949485d1,348,1989-11-08,1990-01-01,55,6.327273
3,00e1d4ba-1ba0-4112-92ea-18f99b8c6b47,21,1989-11-22,1989-12-24,33,0.636364
4,00e65346-d83f-408b-86ef-0bc98beb9b8b,26,1989-11-10,1989-12-31,52,0.500000
...,...,...,...,...,...,...
492,fdbe742b-fd66-4113-b2f2-ed3cc23ed8ad,40,1989-11-10,1989-12-31,52,0.769231
493,fece79a9-64b7-42a5-9338-0125816521ee,118,1989-11-11,1990-01-01,52,2.269231
494,fefe9e7f-4858-4e79-a40c-d6a2b76f351e,116,1989-11-09,1989-12-31,53,2.188679
495,ff09f131-f4ec-4035-b8a2-facad4fefafc,169,1989-11-21,1990-01-01,42,4.023810


In [19]:
#Joning the product level rate of sales dataframe to the inventory data
final_ads=pd.merge(inventory_data,product_sales,left_on='product_variant_id',right_on='product_variant_id',how='left')

In [20]:
final_ads

,product_variant_id,Description,Supplier name,Price,StockOnHand,Sku Classification,Shelf life in days,Lead time,Review time,Qty in Transit,Qty @ Warehouse,total_sales,start_date,end_date,no_days,avg_sales
0,00a4c018-3370-4d6e-8ee9-08d9949485d1,Cut Pineapple 200.0 g,A,54,23,A+,2,2,1,0,0,348.0,1989-11-08,1990-01-01,55.0,6.327273
1,00e65346-d83f-408b-86ef-0bc98beb9b8b,Surf Excel Easy Wash Detergent Powder 1 kg,B,118,6,A,720,4,7,1,0,26.0,1989-11-10,1989-12-31,52.0,0.500000
2,065707f5-d5fc-4c08-a13a-caba8e89a63c,Kissan Fresh Tomato Ketchup 950 g,B,120,39,A,180,4,7,0,0,53.0,1989-11-08,1990-01-01,55.0,0.963636
3,078bde4b-6fd2-4c58-b003-10262868e30e,Neelam Low Sodium Salt Soya Stick 200 g,C,70,7,A,180,2,3,1,0,114.0,1989-11-09,1990-01-01,54.0,2.111111
4,084c1ec1-4447-40c6-90ed-4c1b56d2a72d,Raw Mango 500 g,A,75,48,A,5,2,1,1,0,141.0,1989-11-08,1989-12-31,54.0,2.611111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,7457c7d6-26b9-466b-8950-9c1adb6faa9a,Amul Butter - Pasteurised (School Pack) 10 x 10 g,F,57,52,B,360,4,3,1,0,30.0,1989-11-10,1989-12-05,26.0,1.153846
496,74df3117-b465-4887-902e-cf7de6ba27c9,Gatorade Sports Drink - Lemon Flavor 500 ml,M,50,39,B,270,4,7,0,0,60.0,1989-11-11,1990-01-01,52.0,1.153846
497,750a717b-0f51-4eb1-93d9-f82960cb986d,Baskin-Robbins Very Berry Strawberry Mega Save...,R,395,14,B,210,4,3,0,0,7.0,1989-11-29,1989-12-31,33.0,0.212121
498,75994643-a4e0-4221-846d-80f43703281e,Nissin Cup Mazedaar Masala 70 g,B,50,13,B,240,4,7,30,0,71.0,1989-11-09,1989-12-29,51.0,1.392157


In [21]:
#Taking the maximum of lead time and review time to derive no of days for which order should be placed without OOS at product level 
final_ads['No_days_to_survive']=np.where(final_ads['Lead time ']>final_ads['Review time'],final_ads['Lead time '],final_ads['Review time'])
#Deriving the final  lo of days for which order should be placed by considering shelf life and lead time and taking the least value of both in orderto avoid wastage
final_ads['No_days_to_survive']=np.where(final_ads['No_days_to_survive']>(final_ads['Shelf life in days']+final_ads['Lead time ']),(final_ads['Shelf life in days']+final_ads['Lead time ']),final_ads['No_days_to_survive'])

In [22]:
#Total stock is calculated by summing up the stock in hand, Qty in transit and Qty @ Warehouse
final_ads['total_stock']=final_ads['StockOnHand']+final_ads['Qty in Transit']+final_ads['Qty @ Warehouse']
#safety stock is calculated by considering that A+ SKUs should have 1 day rate of sale of extra total stock 
final_ads['safety_stock_needed']=np.where(np.logical_and((final_ads['Sku Classification']=='A+'),(final_ads['total_stock']<(2*final_ads['avg_sales']))),final_ads['avg_sales'],0)
#total stock needed is calculated by summing the safety stock needed and based on rateof sale and no of days to survive
final_ads['total_stock_needed']=(final_ads['avg_sales']*final_ads['No_days_to_survive'])+final_ads['safety_stock_needed']

In [23]:
#Quantity to order is calculated based on difference between total stock and total stock needed
final_ads['Qty to order']=np.where(final_ads['total_stock_needed']-final_ads['total_stock']<0,0,(final_ads['total_stock_needed']-final_ads['total_stock']))
final_ads['Qty to order'] = final_ads['Qty to order'].apply(np.ceil)

In [24]:
#Selecting the required columns
final_ads=final_ads[['product_variant_id', 'Description',
        'Shelf life in days','avg_sales','StockOnHand',  'Lead time ',
       'Review time', 'Qty in Transit', 'Qty @ Warehouse',
       'Qty to order']]
final_ads.rename(columns={'Shelf life in days':'Shelf life','avg_sales':'Rate of Sales'}, inplace = True)

C:\Users\bsviz\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [25]:
#Writing the csv to attach it later to a email
filename="Replenishment Schedule "+ str(date.today())+'.csv'
final_ads.to_csv(filename,index=False)

In [26]:
#Code to send the file in email
fromaddr = "from email id"
toaddr = ", ".join(['to email ids'])
   
# instance of MIMEMultipart
msg = MIMEMultipart()
  
# storing the senders email address  
msg['From'] = fromaddr
  
# storing the receivers email address 
msg['To'] = toaddr
  
# storing the subject 
msg['Subject'] = "Assignment | Replenishment Schedule |"+ str(date.today())
  
# string to store the body of the mail
body = "Hi All,\nPFA the replenishment schedule for today"
  
# attach the body with the msg instance
msg.attach(MIMEText(body, 'plain'))
  
# open the file to be sent 

attachment = open(filename, "rb")
  
# instance of MIMEBase and named as p
p = MIMEBase('application', 'octet-stream')
  
# To change the payload into encoded form
p.set_payload((attachment).read())
  
# encode into base64
encoders.encode_base64(p)
   
p.add_header('Content-Disposition', "attachment; filename= %s" % filename)
  
# attach the instance 'p' to instance 'msg'
msg.attach(p)
# creates SMTP session
s = smtplib.SMTP('smtp.gmail.com', 587)
  
# start TLS for security
s.starttls()
  
# Authentication
s.login(fromaddr, "your password")
  
# Converts the Multipart msg into a string
text = msg.as_string()
  
# sending the mail
s.sendmail(fromaddr, toaddr, text)
  
# terminating the session
s.quit()

(221, b'2.0.0 closing connection t8sm12007680pgv.35 - gsmtp')